In [2]:
# imports and some initial setup
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf

print(tf.__version__)

emotion_map = {
  0: 'Angry',
  1: 'Disgust',
  2: 'Fear',
  3: 'Happy',
  4: 'Sad',
  5: 'Surprise',
  6: 'Neutral'
}

train_data_raw = pd.read_csv('datasets/fer2013/train.csv')
test_data_raw = pd.read_csv('datasets/fer2013/test.csv')

# create a data generator to not load all images into memory
def data_generator(data, batch_size=32):
  while True:
    for start in range(0, len(data), batch_size):
      end = min(start + batch_size, len(data))
      batch_data = data[start:end]
      batch_features = []
      batch_labels = []
      for _, row in batch_data.iterrows():
        img = np.array(row['pixels'].split(), 'float32').reshape(48, 48)
        img /= 255.0 
        img_expanded = np.expand_dims(img, axis=-1)
        img_rgb = np.repeat(img_expanded, 3, axis=-1)
        batch_features.append(img_rgb)
        batch_labels.append(row['emotion'])
      yield np.stack(batch_features), to_categorical(batch_labels, num_classes=len(emotion_map))

train_generator = data_generator(train_data_raw)


2.17.0


In [17]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import Precision, Recall
import json

# Load the EfficientNetB0 model pre-trained on ImageNet, excluding the top layers
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
predictions = Dense(len(emotion_map), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
  layer.trainable = False

# adding temporary unfreeze for testing
for layer in base_model.layers[-30:]:
  layer.trainable = True

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
  rotation_range=10,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.1,
  zoom_range=0.1,
  horizontal_flip=True,
  brightness_range=[0.8, 1.2],
  fill_mode='nearest'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)

def augmented_data_generator(generator, datagen, batch_size=32):
  while True:
    # Get a batch from the custom generator
    batch_features, batch_labels = next(generator)
    # Use the datagen.flow to augment the batch
    augmented_data = datagen.flow(batch_features, batch_labels, batch_size=batch_size, shuffle=False)
    # Yield the augmented batch
    yield next(augmented_data)

train_generator_augmented = augmented_data_generator(data_generator(train_data_raw), datagen, 64)

# Fit the model using the data generator
history = model.fit(
  train_generator_augmented,
  steps_per_epoch=len(train_data_raw) // 64,
  epochs=30,
  callbacks=[early_stopping, reduce_lr],
  validation_data=augmented_data_generator(data_generator(test_data_raw), datagen),
  validation_steps=len(test_data_raw) // 64,
)


Epoch 1/50
897/897 ━━━━━━━━━━━━━━━━━━━━ 215s 226ms/step - accuracy: 0.1608 - loss: 2.6954 - precision: 0.1805 - recall: 0.0080 - val_accuracy: 0.2680 - val_loss: 2.3995 - val_precision: 0.6061 - val_recall: 0.0056 - learning_rate: 1.0000e-05
Epoch 2/50
897/897 ━━━━━━━━━━━━━━━━━━━━ 194s 217ms/step - accuracy: 0.2253 - loss: 2.4844 - precision: 0.4352 - recall: 0.0208 - val_accuracy: 0.3010 - val_loss: 2.3340 - val_precision: 0.6825 - val_recall: 0.0463 - learning_rate: 1.0000e-05
Epoch 3/50
897/897 ━━━━━━━━━━━━━━━━━━━━ 189s 211ms/step - accuracy: 0.2611 - loss: 2.4093 - precision: 0.5269 - recall: 0.0370 - val_accuracy: 0.3269 - val_loss: 2.2912 - val_precision: 0.6694 - val_recall: 0.0680 - learning_rate: 1.0000e-05
Epoch 4/50
897/897 ━━━━━━━━━━━━━━━━━━━━ 172s 192ms/step - accuracy: 0.2801 - loss: 2.3703 - precision: 0.5752 - recall: 0.0481 - val_accuracy: 0.3497 - val_loss: 2.2541 - val_precision: 0.7118 - val_recall: 0.0750 - learning_rate: 1.0000e-05
Epoch 5/50
897/897 ━━━━━━━━━━━━━

In [ ]:
# model.save('fer2013_efficientNetB3.keras')

test_features = []
test_labels = []

for index, row in test_data_raw.iterrows():
  img = np.array(row['pixels'].split(), 'float32').reshape(48, 48)
  img /= 255.0 
  img_expanded = np.expand_dims(img, axis=-1)
  img_rgb = np.repeat(img_expanded, 3, axis=-1)
  test_features.append(img_rgb)
  test_labels.append(row['emotion'])

test_labels_categorical = to_categorical(test_labels, num_classes=len(emotion_map))

# Evaluate the model
evaluation = model.evaluate(np.array(test_features), test_labels_categorical)
print(f"Test Loss: {evaluation[0]}")
print(f"Test Accuracy: {evaluation[1]}")

test_features = np.array(test_features)
test_labels = np.array(to_categorical(test_labels))

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.preprocessing import label_binarize
from itertools import cycle
import seaborn as sns
import tensorflow as tf

def evaluate_classification_model(model, test_features, test_labels, class_names):
    """
    Evaluates a classification model and generates key metrics and visualizations.
    
    Parameters:
    - model: Trained Keras/TensorFlow model.
    - test_features: Numpy array of test features.
    - test_labels: Numpy array of one-hot encoded test labels.
    - class_names: List of class names corresponding to emotions.
    
    Returns:
    - None (generates visualizations and prints metrics).
    """

    # Predict classes and probabilities
    predictions = model.predict(test_features)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(test_labels, axis=1)
    
    # Generate the classification report for precision, recall, F1 Score
    report = classification_report(true_classes, predicted_classes, target_names=class_names)
    print("Classification Report:\n", report)

    # Confusion Matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

    # ROC Curve and AUC
    n_classes = len(class_names)
    true_classes_binarized = label_binarize(true_classes, classes=range(n_classes))
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    plt.figure(figsize=(10, 8))
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(true_classes_binarized[:, i], predictions[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], label=f'ROC curve of class {class_names[i]} (AUC = {roc_auc[i]:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Each Emotion')
    plt.legend(loc="lower right")
    plt.show()

    # Precision-Recall Curve
    plt.figure(figsize=(10, 8))
    for i in range(n_classes):
        precision, recall, _ = precision_recall_curve(true_classes_binarized[:, i], predictions[:, i])
        plt.plot(recall, precision, label=f'Precision-Recall curve of class {class_names[i]}')
    
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve for Each Emotion')
    plt.legend(loc="lower left")
    plt.show()
    
    # Training/Validation Curves
    history = model.history.history  # Assuming you saved the `history` during training
    plt.figure(figsize=(14, 5))

    # Accuracy Curve
    plt.subplot(1, 2, 1)
    plt.plot(history['accuracy'], label='Training Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')

    # Loss Curve
    plt.subplot(1, 2, 2)
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

emotion_list = [
'Angry',
'Disgust',
'Fear',
'Happy',
'Sad',
'Surprise',
'Neutral'
]

evaluate_classification_model(model, test_features, test_labels, emotion_list)

NameError: name 'model' is not defined

In [ ]:
from tensorflow.keras.preprocessing import image

# predict using the model on a real image
def predict_emotion(img_path):
  img = image.load_img(img_path, target_size=(48, 48), color_mode='grayscale')
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array = np.repeat(img_array, 3, axis=-1)  # Convert grayscale to 3 channels

  prediction = model.predict(img_array)
  predicted_emotion = emotion_map[np.argmax(prediction)]
  
  plt.imshow(img, cmap='gray')
  plt.title(f"Predicted Emotion: {predicted_emotion}")
  plt.axis('off')
  plt.show()

# Example usage
predict_emotion('path_to_your_image.jpg')

In [ ]:
evaluate_classification_model(model, np.array(test_features), test_labels_categorical, class_names)